In [30]:
import pandas as pd
import re
from datetime import datetime, timedelta
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
df1 = pd.read_csv('Webscraped_Data.csv')
df1.shape
df1.head()

,id,name,address,openStatusFormatted,hoursStatusFormatted,latitude,longitude,Hour24_Service
0,1006241,Bath - Avon Street,"Unit B Strahan House Avon Street, Bath",Open until 6:00 PM,Open today from 7:30 AM - 6:00 PM,51.38079,-2.36316,False
1,1009228,Bath Spa Station - Platform 2,"Bath Spa Station Platform 1, Bath",Open until 8:00 PM,Open today from 6:00 AM - 8:00 PM,51.37771,-2.35711,False
2,1030849,Bristol-Gallagher Retail Park,"Gallagher Retail Park Longwell Green, Bristol",Open until 6:00 PM,Open today from 6:30 AM - 6:00 PM,51.44568,-2.49893,False
3,1019405,Beckington - Trowbridge Road A,"A36 Trowbridge Rd, Frome, Frome",Open until 9:00 PM,Open today from 5:30 AM - 9:00 PM,51.26935,-2.28434,False
4,1028511,Bristol-Bath Rd DT,"821 Bath Rd, Bristol",Open until 7:00 PM,Open today from 6:30 AM - 7:00 PM,51.43007,-2.54015,False


Data Preprocessing

In [32]:
# Check for missing values in each column
print(df1.isnull().sum())


id                       0
name                     0
address                  0
openStatusFormatted     41
hoursStatusFormatted     0
latitude                 0
longitude                0
Hour24_Service           0
dtype: int64


In [33]:
#Since the extracted OpenStatusFormatted is less useful in the data analysis, let's drop the column
df = df1.drop('openStatusFormatted',axis=1)

# Check for missing values oncethe column was dropped
print(df.isnull().sum())
df.head()

id                      0
name                    0
address                 0
hoursStatusFormatted    0
latitude                0
longitude               0
Hour24_Service          0
dtype: int64


,id,name,address,hoursStatusFormatted,latitude,longitude,Hour24_Service
0,1006241,Bath - Avon Street,"Unit B Strahan House Avon Street, Bath",Open today from 7:30 AM - 6:00 PM,51.38079,-2.36316,False
1,1009228,Bath Spa Station - Platform 2,"Bath Spa Station Platform 1, Bath",Open today from 6:00 AM - 8:00 PM,51.37771,-2.35711,False
2,1030849,Bristol-Gallagher Retail Park,"Gallagher Retail Park Longwell Green, Bristol",Open today from 6:30 AM - 6:00 PM,51.44568,-2.49893,False
3,1019405,Beckington - Trowbridge Road A,"A36 Trowbridge Rd, Frome, Frome",Open today from 5:30 AM - 9:00 PM,51.26935,-2.28434,False
4,1028511,Bristol-Bath Rd DT,"821 Bath Rd, Bristol",Open today from 6:30 AM - 7:00 PM,51.43007,-2.54015,False


In [34]:
# Remove duplicate entries based on the 'id' column
df = df.drop_duplicates(subset='id')

(1025, 7)

In [35]:
# Extract the last word from the 'address' column and create a new 'starbuck_location' column
df['starbuck_location'] = df['address'].apply(lambda x: x.split()[-1])
df['starbuck_location'].head()

,starbuck_location
0,Bath
1,Bath
2,Bristol
3,Frome
4,Bristol


Converting scraped "hoursStatusFormatted" into meaningful data

In [37]:
# Define the pattern for extraction
pattern = re.compile(r'from (\d{1,2}:\d{2} [APM]{2}) - (\d{1,2}:\d{2} [APM]{2})')

# Function to extract opening and closing hours
def extract_hours(status):
    match = pattern.search(status)
    if match:
        return pd.Series([match.group(1), match.group(2)])
    return pd.Series([None, None])

# Apply the function to the DataFrame
df[['OpeningHour', 'ClosingHour']] = df['hoursStatusFormatted'].apply(extract_hours)


In [38]:
# Function to calculate the duration between opening and closing times
def calculate_duration(row):
    if pd.notna(row['OpeningHour']) and pd.notna(row['ClosingHour']):
        time_format = "%I:%M %p"
        open_time = datetime.strptime(row['OpeningHour'], time_format)
        close_time = datetime.strptime(row['ClosingHour'], time_format)
        if close_time < open_time:
            close_time += timedelta(days=1)  # Account for cases where closing time is after midnight
        duration = close_time - open_time
        return duration.seconds / 3600  # Convert duration to hours
    return 0

# Apply the function to calculate service hours
df['Number_of_Service_Hours'] = df.apply(calculate_duration, axis=1)
# Apply the function to calculate service hours
df['Number_of_Service_Hours'] = df.apply(calculate_duration, axis=1)

#Handling 24hour Starbucks' Missing Values

# Assuming 'Hour24_Service' is a column in the DataFrame
# Convert the 'Hour24_Service' column to boolean for better comparison
df['Hour24_Service'] = df['Hour24_Service'].astype(str).str.upper() == 'TRUE'

# Apply the condition to set values
df.loc[df['Hour24_Service'], 'Number_of_Service_Hours'] = 24
df.loc[df['Hour24_Service'], 'OpeningHour'] = '07:00 AM'
df.loc[df['Hour24_Service'], 'ClosingHour'] = '07:00 AM'
df = df.dropna(subset=['OpeningHour','ClosingHour'])
# Display the DataFrame
df[['OpeningHour', 'ClosingHour', 'Number_of_Service_Hours']].head()

,OpeningHour,ClosingHour,Number_of_Service_Hours
0,7:30 AM,6:00 PM,10.5
1,6:00 AM,8:00 PM,14.0
2,6:30 AM,6:00 PM,11.5
3,5:30 AM,9:00 PM,15.5
4,6:30 AM,7:00 PM,12.5


In [42]:
# Filter out rows with incorrect longitude values
df = df[(df['longitude'] > -60)]
df.shape

(811, 11)

Create a new dataframe for final cleaned dataset

In [43]:
df_clean = df[['id','name','address','starbuck_location','latitude','longitude','OpeningHour','ClosingHour','Hour24_Service','Number_of_Service_Hours']]
df_clean.head()

,id,name,address,starbuck_location,latitude,longitude,OpeningHour,ClosingHour,Hour24_Service,Number_of_Service_Hours
0,1006241,Bath - Avon Street,"Unit B Strahan House Avon Street, Bath",Bath,51.38079,-2.36316,7:30 AM,6:00 PM,False,10.5
1,1009228,Bath Spa Station - Platform 2,"Bath Spa Station Platform 1, Bath",Bath,51.37771,-2.35711,6:00 AM,8:00 PM,False,14.0
2,1030849,Bristol-Gallagher Retail Park,"Gallagher Retail Park Longwell Green, Bristol",Bristol,51.44568,-2.49893,6:30 AM,6:00 PM,False,11.5
3,1019405,Beckington - Trowbridge Road A,"A36 Trowbridge Rd, Frome, Frome",Frome,51.26935,-2.28434,5:30 AM,9:00 PM,False,15.5
4,1028511,Bristol-Bath Rd DT,"821 Bath Rd, Bristol",Bristol,51.43007,-2.54015,6:30 AM,7:00 PM,False,12.5


In [44]:
#Checking for Missing Values in the final dataset
df_clean.isnull().sum()

,0
id,0
name,0
address,0
starbuck_location,0
latitude,0
longitude,0
OpeningHour,0
ClosingHour,0
Hour24_Service,0
Number_of_Service_Hours,0


Saving the final clean dataset

In [45]:
# Save the DataFrame to a CSV file
df_clean.to_csv('Starbucks_Cleaned_Dataset.csv', index=False)

# Download the file
files.download('Starbucks_Cleaned_Dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [46]:
df_lat_long = df[['id','latitude','longitude']]
df_lat_long.head()

# Save the DataFrame to a CSV file
df_lat_long.to_csv('LatLongData.csv', index=False)

# Download the file
files.download('LatLongData.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>